In [ ]:
import sys
# Install gurobi
!{sys.executable} -m pip install -i https://pypi.gurobi.com gurobipy
!{sys.executable} -m pip install plotly geopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB, quicksum

# These are ours
import problem
import visualization

In [ ]:
def normalize(i, j):
    assert i != j
    return (i, j) if i < j else (j, i)

## Problem configuration

In [ ]:
from model import Autonomax, Config

# Run on the first |C| cities (mostly for testing)
C = 41

# What scenario we will be utilizing
scenario = 0
# The number of cities in the core net
NC = 3 
# 1 if the core net should be a cycle; 0 if it shoul be a path.
Z = 1

autonomax = Autonomax(
    Config(
        cities=problem.cities[:C],
        distances=problem.D[:C, :C],
        demand=problem.B[scenario][:C],
        core_city_count=NC,
        core_net_is_cycle=Z,
    )
)

In [ ]:
A = autonomax.model.getA()

count = lambda d: len(d) if isinstance(d, gp.tupledict) else 1

V = sum(count(v) for v in autonomax.variables.values())
C = sum(count(c) for c in autonomax.constraints.values())

print(f'V = {V}, C = {C}')

fig, ax = plt.subplots(1, figsize=(128, 128 * C/V))
ax.set_xticklabels([])
ax.set_yticklabels([])
plt.spy(A)


total = 0
print(f'\nCONSTRAINT SETS:')
for (name, constraint) in autonomax.constraints.items():
    constraints_in_set = count(constraint)
    print(f'{constraints_in_set:>5} | {name}')
    #plt.gcf().text(0.07, 0.69 - 0.4 * (total + 0.5 * count(constraint)) / C, name, fontsize=14)
    total += constraints_in_set
    plt.plot([0, V], [total - 0.5, total - 0.5], color='grey')
    
total = 0
print(f'\nVARIABLE SETS:')
for (name, variables) in autonomax.variables.items():
    variables_in_set = count(variables)
    print(f'{variables_in_set:>5} | {name}')
    #plt.gcf().text(0.1 + 0.8 * (total + 0.5*count(variables)) / V, 0.8, name, fontsize=14)
    total += count(variables)
    plt.plot([total - 0.5, total - 0.5], [0, C], color='grey')
    
plt.savefig('constraint-matrix.png', dpi=200)

In [ ]:
autonomax.model.Params.timeLimit = 600.0
autonomax.model.optimize()

## Solution

In [ ]:
pd.DataFrame(autonomax.city_info())

In [ ]:
pd.DataFrame(autonomax.edge_info())

In [ ]:
visualization.show(pd.DataFrame(autonomax.city_info()), pd.DataFrame(autonomax.edge_info()))